In [ ]:
from google.colab import auth
auth.authenticate_user()

!git config --global user.email "rizkynaufal552@gmail.com"
!git config --global user.name "Rizky Naufal"

!git clone https://@github.com/IET-Polinela/ujian-tengah-semester-RizkynaufalF

Cloning into 'ujian-tengah-semester-RizkynaufalF'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [6]:
%cd /content/ujian-tengah-semester-RizkynaufalF

/content/ujian-tengah-semester-RizkynaufalF


In [30]:
%%writefile Preprocessing.py
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df = df.drop(['id'], axis=1)

# Perbaikan warning: isi nilai null di 'bmi' tanpa inplace
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())

# Encode fitur kategorikal
le = LabelEncoder()
for col in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    df[col] = le.fit_transform(df[col])

# Split fitur dan target
X = df.drop('stroke', axis=1)
y = df['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print info hasil preprocessing
print("Preprocessing selesai.")
print("Jumlah data latih:", X_train.shape[0])
print("Jumlah data uji:", X_test.shape[0])
print("Contoh data fitur:\n", X_train.head())

# Simpan hasil preprocessing ke dalam CSV
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False, header=True)
y_test.to_csv('y_test.csv', index=False, header=True)

Overwriting Preprocessing.py


In [31]:
!python Preprocessing.py

Preprocessing selesai.
Jumlah data latih: 4088
Jumlah data uji: 1022
Contoh data fitur:
       gender   age  hypertension  ...  avg_glucose_level   bmi  smoking_status
802        1  79.0             0  ...             112.64  28.5               1
3927       0  62.0             0  ...              88.32  36.3               0
2337       0  21.0             0  ...              59.52  33.7               2
3910       1  31.0             0  ...              65.70  30.4               1
1886       0  31.0             0  ...              59.63  19.9               2

[5 rows x 10 columns]


In [47]:
%%writefile XGBoost.py
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from xgboost import XGBClassifier

# Load hasil preprocessing
X_train = pd.read_csv('X_train.csv')
X_test  = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()  # Ensure y_train is 1D
y_test  = pd.read_csv('y_test.csv').values.ravel()    # Ensure y_test is 1D

# ---------- 1. XGBoost Classifier ----------
# Calculate the scale_pos_weight for handling class imbalance
scale_pos_weight = len(y_train) / sum(y_train == 1)

# Inisialisasi dan training
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, scale_pos_weight=scale_pos_weight)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

# Evaluasi XGBoost
print("=== XGBoost Classification Report ===")
print(classification_report(y_test, y_pred_xgb))

# Confusion Matrix XGBoost
disp_xgb = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred_xgb, cmap='Blues', normalize=None
)
disp_xgb.ax_.set_title('Confusion Matrix - XGBoost')
plt.savefig('confusion_matrix_xgb.png')
plt.close()

# Feature Importance XGBoost
i_importances = xgb.feature_importances_
feat_names = X_train.columns  # Assuming X_train is a DataFrame with columns
imp_xgb = pd.Series(i_importances, index=feat_names).sort_values()
plt.figure(figsize=(8,6))
imp_xgb.plot(kind='barh')
plt.title('Feature Importance - XGBoost')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.savefig('feature_importance_xgb.png')
plt.close()


Overwriting XGBoost.py


In [48]:
!python XGBoost.py

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:09:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
=== XGBoost Classification Report ===
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       960
           1       0.28      0.21      0.24        62

    accuracy                           0.92      1022
   macro avg       0.62      0.59      0.60      1022
weighted avg       0.91      0.92      0.91      1022



In [49]:
!git add .
!git commit -m "UTS"
!git push origin main

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> ujian-tengah-semester-RizkynaufalF
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached ujian-tengah-semester-RizkynaufalF
hint: 
hint: See "git help submodule" for more information.
[main f77a342] UTS
 10 files changed, 15411 insertions(+)
 create mode 100644 Preprocessing.py
 create mode 100644 XGBoost.py
 create mode 100644 X_test.csv
 create mode 100644 X_train.csv
 create mode 100644 confusion_matrix_xgb.png
 create mode 100644 feature_importance_xgb.png
 create mode 100644 healthcare-dataset-stroke-data.csv
 create mode 160000 ujian-tengah-semester-RizkynaufalF
 create mode 100644 y_test.csv
 create mode 100644 y_train.csv
E

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
!cp "/content/drive/My Drive/Colab Notebooks/UTS_23758025.ipynb" "ujian-tengah-semester-RizkynaufalF"

In [55]:
!git add .
!git commit -m "upload Final"
!git push origin main

[main 34b2e5c] upload Final
 1 file changed, 1 insertion(+)
 create mode 100644 UTS_23758025.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.69 KiB | 3.69 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:  